In [1]:
import json
import collections
import numpy as np
import scipy.stats

num_prompts = 27
seeds = [0, 42, 23]
model1 = "deberta-v3-base"
model2 = "deberta-v3-large"
rte_model_address = "./experiments/rte/{}/supervised/"
setup_name = "32shots-p{}-i0-seed{}/"
t5_base_rte = np.array([[], [], []])
t5_11b_rte = np.array([[], [], []])
def read_accuracy(address, model, seedID, promptID):
    if model == "T5-base":
        return t5_base_rte[seedID][promptID]
    elif model == "T5-11B":
        return t5_11b_rte[seedID][promptID]
    else:
        with open(address + "results.json") as f:
            tmp = json.load(f)
        return tmp['dev_set_after_training']['acc']
models = ["roberta-large", "roberta-base", "deberta-v3-base", "deberta-v3-large", "T5-base", "T5-11B"]
model_combs = []
for i in range(len(models)):
    for j in range(i+1, len(models)):
        model_combs.append((models[i], models[j]))

def H(p, q):
  # distance between p and d
  # p and q are np array probability distributions
  n = len(p)
  sum = 0.0
  for i in range(n):
    sum += (np.sqrt(p[i]) - np.sqrt(q[i]))**2
  result = (1.0 / np.sqrt(2.0)) * np.sqrt(sum)
  return result

In [2]:
# Accuracy
for model_pair in model_combs:
    model1_dev_acc, model1_test_acc, model2_dev_acc, model2_test_acc = [], [], [], []
    model1 = model_pair[0]
    model2 = model_pair[1]
    for i in range(num_prompts):
        avg_dev1_acc, avg_dev2_acc = 0, 0
        for seedID, seed in enumerate(seeds):
            dev1_acc = read_accuracy(rte_model_address.format(model1) + setup_name.format(i, seed), model1, seedID, i)
            dev2_acc = read_accuracy(rte_model_address.format(model2) + setup_name.format(i, seed), model2)
            avg_dev1_acc += dev1_acc
            avg_dev2_acc += dev2_acc
        model1_dev_acc.append(avg_dev1_acc / len(seeds))
        model2_dev_acc.append(avg_dev2_acc / len(seeds))
    # pearsonr
    pearsonr = (scipy.stats.pearsonr(model1_dev_acc, model2_dev_acc))
    # spearmanr
    spearmanr = (scipy.stats.spearmanr(model1_dev_acc, model2_dev_acc))
    # kendalltau
    kendalltau = (scipy.stats.kendalltau(model1_dev_acc, model2_dev_acc))
    h_distance = (h_distance(model1_dev_acc, model2_dev_acc), h_distance(model2_dev_acc, model1_dev_acc))
    print(f"{model1}, {model2} :\n pearsonr:{pearsonr} \n spearmanr: {spearmanr} \n kendalltau: {kendalltau} \n Hellinger distance: {h_distance}")

SyntaxError: invalid syntax (1875304390.py, line 12)

((0.8574021489638177, 0.0015174361481669982),
 (0.8097377768954448, 0.004525084592242043))

(SpearmanrResult(correlation=0.8333492104318903, pvalue=0.0027453959942271722),
 SpearmanrResult(correlation=0.7859363967596924, pvalue=0.007030735401842157))

(KendalltauResult(correlation=0.7059312080025176, pvalue=0.006056273869219234),
 KendalltauResult(correlation=0.6897007348075543, pvalue=0.006616023243112296))